In [ ]:
import shutil
from experiment import Experiment

In [ ]:
# Unzip files
# shutil.unpack_archive("data_export.zip", "data")
# shutil.unpack_archive("error_correcting_codes.zip", "error_correcting_codes")

In [ ]:
# Process files to create an Experiment
experiment = Experiment.reconstruct_experiment_from_folder("data/hamming-4-oslo/hamming-4-nairobi-8-n-40-tau-0")

print(experiment.get_experiment_info())

baseline_experiment = None
# baseline_experiment = Experiment.reconstruct_experiment_from_folder("data/hamming-4-oslo/hamming-4-oslo-8-n-40-tau-0")
if baseline_experiment:
    print("\nBaseline experiment:")
    print(baseline_experiment.get_experiment_info())

In [ ]:
# Test 1 - honest delete
print(experiment.run_test_1())

if baseline_experiment:
    print("\nBaseline experiment:")
    print(baseline_experiment.run_test_1())

In [ ]:
# Test 2 - decrypt
print(experiment.run_test_2())

if baseline_experiment:
    print("\nBaseline experiment:")
    print(baseline_experiment.run_test_2())

In [ ]:
# Test 3 - honest delete and then decrypt
print(experiment.run_test_3())

if baseline_experiment:
    print("\nBaseline experiment:")
    print(baseline_experiment.run_test_3())

In [ ]:
# Test 4 - malicious delete and then decrypt
print(experiment.run_test_4())

if baseline_experiment:
    print("\nBaseline experiment:")
    print(baseline_experiment.run_test_4())

In [ ]:
# Test 5 - tamper detection
print(experiment.run_test_5())

if baseline_experiment:
    print("\nBaseline experiment:")
    print(baseline_experiment.run_test_5())

In [ ]:
# Function to show results of error correction
from decryption_circuit import decrypt_single_result
from utils import hamming_distance
from states import Basis

def display_error_correction_results(exp: Experiment) -> None:
    """Prints the Hamming distances of the computational basis measurements, along with the successful decryptions per Hamming distance."""
    s = set(i for i in range(len(exp.encoded_in_qubits)) if exp.key.theta[i] is Basis.COMPUTATIONAL)
    arr = []
    for meas, count in exp.decryption_counts_test2.items():
        success, _ = decrypt_single_result(meas, exp.key, exp.ciphertext, exp.message, exp.scheme_parameters, error_correct=True)
        decrypt_measurement = "".join([meas[i] for i in range(len(meas)) if i in s])
        expected_decrypt = "".join([exp.encoded_in_qubits[i] for i in range(len(exp.encoded_in_qubits)) if i in s])
        distance = hamming_distance(decrypt_measurement, expected_decrypt)
        arr.extend(((distance, success),) * count)

    for dist in range(max([tup[0] for tup in arr])+1):
        filtered = [tup for tup in arr if tup[0] == dist]
        success_count = [tup[1] for tup in filtered].count(True)
        failure_count = [tup[1] for tup in filtered].count(False)
        if success_count > 0 or failure_count > 0:
            print(f"Distance {dist}: {success_count} successful, {failure_count} unsuccessful, {success_count/(success_count+failure_count)*100} percent")

In [ ]:
display_error_correction_results(experiment)

if baseline_experiment:
    print("\nBaseline experiment:")
    display_error_correction_results(baseline_experiment)

In [ ]:
def get_average_error_rate(exp: Experiment) -> float:
    """Returns the average error rate for measurements in the computational basis."""
    tot = 0
    s = set(i for i in range(len(exp.encoded_in_qubits)) if exp.key.theta[i] is Basis.COMPUTATIONAL)
    for meas in exp.decryption_counts_test2:
        tot += hamming_distance("".join([meas[i] for i in range(len(meas)) if i in s]), "".join([exp.encoded_in_qubits[i] for i in range(len(exp.encoded_in_qubits)) if i in s]))/exp.scheme_parameters.s
    return tot/exp.experiment_properties.shots

In [ ]:
print(get_average_error_rate(experiment))

if baseline_experiment:
    print("\nBaseline experiment:")
    print(get_average_error_rate(baseline_experiment))